In [3]:
from bs4 import BeautifulSoup
import pandas as pd
from tqdm.notebook import tqdm
from urllib.request import Request
import requests
from tqdm import tqdm
import time
from urllib.request import Request
from user_agent import generate_user_agent
from urllib.request import urlopen

In [ ]:
#한국 거래소에서 코스피 코스닥 종목명, 종목코드 가져오기
kospi_df = pd.read_csv("../practice/../../../data/eda_project/KOSPI.csv", encoding= "euc-kr")
kosdaq_df = pd.read_csv("../practice/../../../data/eda_project/KOSDAQ.csv", encoding= "euc-kr")

columns = ["종목코드", "종목명"]

kospi_df = kospi_df[columns]
kospi_df.head()
kosdaq_df = kosdaq_df[columns]

korea_df = pd.concat([kosdaq_df, kospi_df], axis = 0)
korea_df

#네이버 증권에서 테마 크롤
company = []
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.76 Safari/537.36', "Upgrade-Insecure-Requests": "1","DNT": "1","Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8","Accept-Language": "en-US,en;q=0.5","Accept-Encoding": "gzip, deflate"}

for pagenum in tqdm(range(1,9)):
    url = "https://finance.naver.com/sise/theme.naver?&page={pagenum}".format(pagenum=pagenum)
    resp = requests.get(url, headers = headers)
    soup = BeautifulSoup(resp.content, "html.parser")

    s = soup.find_all("td", "col_type1")
    thema_name = []
    thema_link = []
    for i in s:
        a_tag = i.find("a", href=True)
        if a_tag:
            href_value = a_tag["href"]
            thema_link.append('https://finance.naver.com' + href_value)
            thema_name.append(a_tag.string)

    for idx, t in tqdm(enumerate(thema_link)):
        req = Request(url=t, headers={'User-Agent':generate_user_agent()})
        page = urlopen(req)
        soup2 = BeautifulSoup(page, "html.parser")
        
        com_list = soup2.find_all("td", "name")

        for j in range(len(com_list)):
            com = com_list[j].find("a").string

            company.append([com, thema_name[idx]])


df = pd.DataFrame(company)
df.columns = ["종목명", "테마"]


#종목명을 기준으로 한국거래소, 크롤링 데이터 합치기
thema_stock = pd.merge(df, korea_df, on='종목명' , how='outer')

thema_stock = thema_stock[thema_stock["테마"].notna()]


thema_stock = thema_stock[["종목코드", "종목명", "테마"]]

thema_stock.to_csv("../../../data/eda_project/thema.csv")

#테마에 맞는 종목코드 가져오기 일단은 로봇 테마만
robot_thema = ["로봇(산업용/협동로봇 등)", "지능형로봇/인공지능(AI)"]
robot_stock = thema_stock[thema_stock["테마"].isin(robot_thema)]
robot_stock_df = robot_stock[["종목코드", "종목명"]].drop_duplicates()

robot_thema = ["로봇(산업용/협동로봇 등)", "지능형로봇/인공지능(AI)"]
robot_stock = thema_stock[thema_stock["테마"].isin(robot_thema)]
robot_stock_series = robot_stock[["종목코드", "종목명"]].drop_duplicates()

code_unique = robot_stock["종목코드"].unique()

#종목 코드에 맞는 종목명 뽑기
for code in code_unique:
    a = robot_stock_df.loc[robot_stock_df["종목코드"] == code, "종목명"]
    print(a.values[0])




In [ ]:
import FinanceDataReader as fdr
import pandas as pd
    
class Stock_Data:
    def __init__(self, code, start, end):
        self.code = str(code)
        self.start = str(start)
        self.end = str(end)
        self.stock = fdr.DataReader(self.code, self.start, self.end)
        
    def get_data(self):
        # 삼성 종목코드 2023년 주식 데이터
        return self.stock
    
    def get_diff(self):
        # 이전 행과 현재 행 비교(증감 추이 분석)
        return self.stock['Close'].diff() 

    def get_pct(self):
    # 증감 추이 퍼센티지
        return self.stock['Close'].pct_change()
    
    def count_bull(self):
        a1 = self.stock["Close"].pct_change() < 0
        return a1.sum()
        
    def count_bear(self):
        a1 = self.stock["Close"].pct_change() > 0
        return a1.sum()
    
def main():
    data_list = []
    for code in code_unique:
        name = robot_stock_df.loc[robot_stock_df["종목코드"] == code, "종목명"]
        print(f"종목명 : {name.values[0]}")
        data = Stock_Data(code, "2023.01.01", "2023.01.31")
        data_list.append(data)
        print(data.get_data())
    

if __name__ == "__main__":
    main()